In [1]:
!pip install -U -q PyDrive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import pandas as pd
import numpy as np

In [5]:
black = np.array(drive.CreateFile({'id': '1tcyQFsii3ZSHKUhZqoODeFVt97GJQZIN'}).GetContentString().split('\n'))
white = np.array(drive.CreateFile({'id': '1rZbVHZEf5P4fQJovcQazHZqGMXpl1vsd'}).GetContentString().split('\n'))

In [6]:
data = drive.CreateFile({'id': '1-NMkNj-PZAAWEGeoYwr6agwM8RkYhdB8'}).GetContentString().split('\n')

In [7]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        6.8G        505M        249M        5.4G        5.4G
Swap:            0B          0B          0B


In [8]:
def num_states(data):
  res = 0
  for game in data:
    res += len(game.split()) - 1
  return res / 2

In [9]:
num_states(black)

17750698.0

# Splitting black and white

In [11]:
black = []
white = []
size = 0
for line in raw_data:
    game = line.split()
    if (len(game) > 0):
      if (game[0] == 'black'):
          black.append(' '.join(game[1:]))
      elif (game[0] == 'white'):
          white.append(' '.join(game[1:]))
#black = np.array(black)
#white = np.array(white)

# Models saver:

In [12]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [13]:
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

def saver(name):
  file_metadata = {
    'name': name + '.h5',
    'mimeType': 'application/octet-stream'
  }
  media = MediaFileUpload(name + '.h5', 
                          mimetype='application/octet-stream',
                          resumable=True)
  created = drive_service.files().create(body=file_metadata,
                                         media_body=media).execute()
  print('File ID: {}'.format(created.get('id')))

# Generators for Networks:

In [14]:
POS_TO_LETTER = 'abcdefghjklmnop'
LETTER_TO_POS = {letter: pos for pos, letter in enumerate(POS_TO_LETTER)}

from random import randint, shuffle

def to_pos(move):
    return int(move[1:]) - 1, LETTER_TO_POS[move[0]]  
  
def rotare(img, times):
    if times == 0:
      return img
    if times == 1:
      return np.rot90(img)
    if times == 2:
      return np.rot90(np.rot90(img))
    if times == 3:
      return np.rot90(np.rot90(np.rot90(img)))

def generator(data, batch_size = 64):
    datas = []
    labels = []
    size = 0
    while(1):
      try:
        for game in data:
          img = np.zeros((15, 15))
          game = game.split(' ')
          min_i, min_j, max_i, max_j = 15, 15, 0, 0
          for move in range(1, len(game)):
              i, j = to_pos(game[move])
              min_i = min(min_i, i)
              min_j = min(min_j, j)
              max_i = max(max_i, i)
              max_j = max(max_j, j)
              move_i = 0
              move_j = 0
              if (randint(0, 1)):
                if (randint(0, 1)):
                  move_i = randint(0, 15 - max_i)
                  move_j = randint(0, 15 - max_j)
                else:
                  move_i = randint(0, 15 - max_i)
                  move_j = -randint(0, min_j)
              else:
                if (randint(0, 1)):
                  move_i = -randint(0, min_i)
                  move_j = randint(0, 15 - max_j)
                else:
                  move_i = -randint(0, min_i)
                  move_j = -randint(0, min_j)

              if (move % 2 == 1):
                if (game[0] == 'black'):
                  rolled = np.roll(np.roll(img, move_i, axis = 0), move_j, axis = 1)
                  label = np.zeros((15, 15))
                  label[i, j] = 1
                  label = np.roll(np.roll(label, move_i, axis = 0), move_j, axis = 1)
                  seed = randint(0, 3)
                  rolled = rotare(rolled, seed)
                  res = np.zeros((1, 15, 15, 4))
                  for i in range(15):
                    for j in range(15):
                      if (rolled[i, j] == 1):
                        res[0, i, j, 0] = 1
                        res[0, i, j, 2] = 1
                      elif (data[i, j] == -1):
                        res[0, i, j, 1] = 1
                        res[0, i, j, 2] = 1
                      res[0, i, j, 3] = 1
                  datas.append(res)
                  labels.append(rotare(label, seed).flatten())
                  size += 1
                img[i][j] = 1
              else:
                if (game[0] == 'white'):
                  rolled = np.roll(np.roll(img, move_i, axis = 0), move_j, axis = 1)
                  label = np.zeros((15, 15))
                  label[i, j] = 1
                  label = np.roll(np.roll(label, move_i, axis = 0), move_j, axis = 1)
                  seed = randint(0, 3)
                  rolled = rotare(rolled, seed)
                  res = np.zeros((1, 15, 15, 4))
                  for i in range(15):
                    for j in range(15):
                      if (rolled[i, j] == 1):
                        res[0, i, j, 0] = 1
                        res[0, i, j, 2] = 1
                      elif (data[i, j] == -1):
                        res[0, i, j, 1] = 1
                        res[0, i, j, 2] = 1
                      res[0, i, j, 3] = -1
                  datas.append(res)
                  labels.append(rotare(label, seed).flatten())
                  size += 1
                img[i][j] = -1

              if (size == batch_size):
                yield np.array(datas).reshape((batch_size, 15, 15, 4)), np.array(labels).reshape((batch_size, 225))
                datas = []
                labels = []
                size = 0
      except GeneratorExit:
        raise GeneratorExit('exiting...')
      except:
        continue

def generator_small(data, batch_size = 64):
    datas = []
    labels = []
    size = 0
    while(1):
      random.shuffle(data)
      try:
        for game in data:
          img = np.zeros((15, 15))
          game = game.split(' ')
          min_i, min_j, max_i, max_j = 15, 15, 0, 0
          for move in range(1, len(game)):
              i, j = to_pos(game[move])
              min_i = min(min_i, i)
              min_j = min(min_j, j)
              max_i = max(max_i, i)
              max_j = max(max_j, j)
              move_i = 0
              move_j = 0
              if (randint(0, 1)):
                if (randint(0, 1)):
                  move_i = randint(0, 15 - max_i)
                  move_j = randint(0, 15 - max_j)
                else:
                  move_i = randint(0, 15 - max_i)
                  move_j = -randint(0, min_j)
              else:
                if (randint(0, 1)):
                  move_i = -randint(0, min_i)
                  move_j = randint(0, 15 - max_j)
                else:
                  move_i = -randint(0, min_i)
                  move_j = -randint(0, min_j)

              if (move % 2 == 1):
                if (game[0] == 'black'):
                  rolled = np.roll(np.roll(img, move_i, axis = 0), move_j, axis = 1)
                  label = np.zeros((15, 15))
                  label[i, j] = 1
                  label = np.roll(np.roll(label, move_i, axis = 0), move_j, axis = 1)
                  seed = randint(0, 3)
                  datas.append(rotare(rolled, seed))
                  labels.append(rotare(label, seed).flatten())
                  size += 1
                img[i][j] = 1
              else:
                if (game[0] == 'white'):
                  rolled = np.roll(np.roll(img, move_i, axis = 0), move_j, axis = 1)
                  label = np.zeros((15, 15))
                  label[i, j] = 1
                  label = np.roll(np.roll(label, move_i, axis = 0), move_j, axis = 1)
                  seed = randint(0, 3)
                  datas.append(rotare(rolled, seed))
                  labels.append(rotare(label, seed).flatten())
                  size += 1
                img[i][j] = -1

              if (size == batch_size):
                yield np.array(datas).reshape((batch_size, 15, 15, 1)), np.array(labels).reshape((batch_size, 225))
                datas = []
                labels = []
                size = 0
      except GeneratorExit:
        raise GeneratorExit('exiting...')
      except:
        continue

In [15]:
import random 
def generator_value(data, batch_size = 64, color = 'black'):
    datas = []
    labels = []
    size = 0
    while(1):
      random.shuffle(data)
      try:
        for game in data:
          img = np.zeros((15, 15))
          game = game.split(' ')
          if (len(game) < 5):
            continue
          #selected = set(game[-6:])
          min_i, min_j, max_i, max_j = 15, 15, 0, 0
          for move in range(1, len(game)):
              i, j = to_pos(game[move])
              min_i = min(min_i, i)
              min_j = min(min_j, j)
              max_i = max(max_i, i)
              max_j = max(max_j, j)

              if (move > len(game) - 17 and randint(0, 3) == 3):
                move_i = 0
                move_j = 0
                if (randint(0, 1)):
                  if (randint(0, 1)):
                    move_i = randint(0, 15 - max_i)
                    move_j = randint(0, 15 - max_j)
                  else:
                    move_i = randint(0, 15 - max_i)
                    move_j = -randint(0, min_j)
                else:
                  if (randint(0, 1)):
                    move_i = -randint(0, min_i)
                    move_j = randint(0, 15 - max_j)
                  else:
                    move_i = -randint(0, min_i)
                    move_j = -randint(0, min_j)

                rolled = np.roll(np.roll(img, move_i, axis = 0), move_j, axis = 1)
                seed = randint(0, 3)
                datas.append(rotare(rolled, seed))
                size += 1
                if (game[0] == color):
                  labels.append(1)
                else:
                  labels.append(-1)

              if (move % 2 == 1):
                img[i][j] = 1
              else:
                img[i][j] = -1

              if (size == batch_size):
                yield np.array(datas).reshape((batch_size, 15, 15, 1)), np.array(labels).reshape((batch_size, 1))
                datas = []
                labels = []
                size = 0
      except GeneratorExit:
        raise GeneratorExit('exiting...')
      except:
        continue

In [16]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Activation, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

Using TensorFlow backend.


# Policy Network

## Rollout model:

In [17]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same',
                     input_shape=(15, 15, 1)))
model.add(LeakyReLU())
model.add(Conv2D(16, (2, 2), padding='same'))
model.add(LeakyReLU())
#model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
#model.add(LeakyReLU())
#model.add(BatchNormalization(axis = 3))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(32, (2, 2), padding='same'))
model.add(LeakyReLU())
#model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
#model.add(LeakyReLU())
#model.add(BatchNormalization(axis = 3))
    
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(64, (2, 2), padding='same'))
model.add(LeakyReLU())
#model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
#model.add(LeakyReLU())
#model.add(BatchNormalization(axis = 3))

    
model.add(Conv2D(1, (1, 1), padding='same'))
#model.add(LeakyReLU())
#model.add(Flatten())

#model.add(LeakyReLU())
model.add(Flatten())
#model.add(Dense(1))
#model.add(Dropout(0.35))
#model.add(Dense(225))
    
model.add(Activation('softmax'))

# initiate RMSprop optimizer
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 15, 15, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 16)        1040      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 32)        4640      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 32)        4128      
__________

## Main model

In [18]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same',
                     input_shape=(15, 15, 1)))
model.add(LeakyReLU())
model.add(Conv2D(16, (2, 2), padding='same'))
model.add(LeakyReLU())
#model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
#model.add(LeakyReLU())
#model.add(BatchNormalization(axis = 3))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(32, (2, 2), padding='same'))
model.add(LeakyReLU())
#model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
#model.add(LeakyReLU())
#model.add(BatchNormalization(axis = 3))
    
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(64, (2, 2), padding='same'))
model.add(LeakyReLU())
#model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
#model.add(LeakyReLU())
#model.add(BatchNormalization(axis = 3))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(128, (2, 2), padding='same'))
model.add(LeakyReLU())
#model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
#model.add(LeakyReLU())

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(256, (2, 2), padding='same'))
model.add(LeakyReLU())
#model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
#model.add(LeakyReLU())
#model.add(BatchNormalization(axis = 3))

    
model.add(Conv2D(1, (1, 1), padding='same'))
#model.add(LeakyReLU())
#model.add(Flatten())

#model.add(LeakyReLU())
model.add(Flatten())
#model.add(Dense(1))
#model.add(Dropout(0.35))
#model.add(Dense(225))
    
model.add(Activation('softmax'))

# initiate RMSprop optimizer
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 15, 15, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 16)        1040      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 32)        4640      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 32)        4128      
__________

# BIG PAPA MODEL

In [19]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Activation
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

Using TensorFlow backend.


In [20]:
from keras.models import Model
from keras.optimizers import Nadam
from keras.layers import BatchNormalization, Convolution2D, Input, merge, LeakyReLU
from keras.layers.core import Activation, Layer

def conv_block_3(feat_maps_out, prev):
    prev = Conv2D(feat_maps_out, (3, 3), padding='same')(prev) 
    prev = LeakyReLU()(prev)
    prev = Conv2D(feat_maps_out, (2, 2), padding='same')(prev) 
    prev = LeakyReLU()(prev)
    return prev

def skip_block(feat_maps_in, feat_maps_out, prev):
    if feat_maps_in != feat_maps_out:
        prev = Conv2D(feat_maps_out, (1, 1), padding='same')(prev)
    return prev 

def Residual_3(feat_maps_in, feat_maps_out, prev_layer):
    skip = skip_block(feat_maps_in, feat_maps_out, prev_layer)
    conv = conv_block_3(feat_maps_out, prev_layer)
    return merge([skip, conv], mode='sum')

img_rows = 15 
img_cols = 15 

inp = Input((img_rows, img_cols, 1))
cnv1 = Conv2D(32, (3, 3), padding='same')(inp)
cnv1 = LeakyReLU()(cnv1)
r1 = Residual_3(32, 64, cnv1)
r2 = Residual_3(64, 64, r1)
r3 = Residual_3(64, 128, r2)
r4 = Residual_3(128, 256, r3)
cnv2 = Conv2D(1, (1, 1), padding='same')(r4)
cnv2 = LeakyReLU()(cnv2)
cnv2 = Flatten()(cnv2)
out = Activation('softmax')(cnv2)
model = Model(input=inp, output=out)
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 15, 15, 1)    0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 15, 15, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_21 (LeakyReLU)      (None, 15, 15, 32)   0           conv2d_26[0][0]                  
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 15, 15, 64)   18496       leaky_re_lu_21[0][0]             
__________________________________________________________________________________________________
leaky_re_l

# Value Network:

In [21]:
model = Sequential()
model.add(ZeroPadding2D(padding=(2, 2), input_shape=(15, 15, 1)))
model.add(Conv2D(16, (5, 5),
                        padding='valid'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Conv2D(16, (5, 5),
                        padding='valid')) # same
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(512))  # 1024
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(1))
model.add(Activation('tanh'))

model.compile(loss='mse',
              optimizer="adadelta", metrics = ['accuracy'])

# Fitting value:

In [22]:
epochs = 4
for _ in range(epochs):
  model.fit_generator(generator_value(data), steps_per_epoch = 50000, epochs = 1)
  model.save('value_black.h5')
  saver('value_black')

50000/50000 [==============================] - 1040s 21ms/step - loss: 0.9346 - acc: 0.0598
File ID: 1g7BsA6MIOXdiqIsW4nb2WI1FNMx1Fm7b
50000/50000 [==============================] - 1040s 21ms/step - loss: 0.9133 - acc: 0.0953
File ID: 18YWLn9L6pOAyvYFmlU73rsbmccKrMImj
50000/50000 [==============================] - 1037s 21ms/step - loss: 0.9083 - acc: 0.1052
File ID: 1fNenL4UvUA2Q3qnRtnkt5QzFnb-f6myU
50000/50000 [==============================] - 1031s 21ms/step - loss: 0.9056 - acc: 0.1100
File ID: 1D-nIJdBLSLGB5l2rgzpPRpABAiW91M4d


In [23]:
model.save('value_black256.h5')
saver('value_black256')

File ID: 16BIhkk780AbDTuG9Qlhgs1D8yBvBEfGT


In [24]:
epochs = 4
for _ in range(epochs):
  model.fit_generator(generator_value(data, 256), steps_per_epoch = 50000, epochs = 1)
  model.save('value_black256.h5')
  saver('value_black256')

50000/50000 [==============================] - ETA: 0:00 - loss: 0.8866 - acc: 0.1298

In [0]:
epochs = 4
for _ in range(epochs):
  model.fit_generator(generator_value(data, 1024), steps_per_epoch = 50000, epochs = 1)
  model.save('value_black1024.h5')
  saver('value_black1024')

In [0]:
epochs = 5
for _ in range(epochs):
  model.fit_generator(generator_value(data, 4096), steps_per_epoch = 50000, epochs = 1)
  model.save('value_black4096.h5')
  saver('value_black4096')

# Fitting policy network

In [25]:
from keras.models import load_model
drive.CreateFile({'id': '191FOnjZpHzWkgqU2c1eOxuUuC-qqzM6Z'}).GetContentFile('white1024.h5')
model = load_model('white1024.h5')

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [26]:
model.fit_generator(generator_small(black, batch_size = 64), steps_per_epoch = num_states(black) / 128, epochs = 1)
model.save('black.h5')
saver('black')


Epoch 1/1
138677/138677 [==============================] - ETA: 0s - loss: 2.3617 - acc: 0.3807

In [27]:
model.fit_generator(generator_small(black, batch_size = 128), steps_per_epoch = num_states(black) / 256, epochs = 1)
model.save('black.h5')
saver('black')
#model.fit_generator(generator_small(black, batch_size = 128), steps_per_epoch = num_states(black) / 256, epochs = 1)
#model.save('black.h5')
#saver('black')

69339/69338 [==============================] - 6836s 99ms/step - loss: 2.0352 - acc: 0.4700
File ID: 1_AMGZhdETlhlj-7bdrcAixFw3cWtWYf5


In [28]:
model.fit_generator(generator_small(black, batch_size = 256), steps_per_epoch = num_states(black) / 256, epochs = 1)
model.save('black256.h5')
saver('black256')

Epoch 1/1
69338/69338 [==============================] - ETA: 0s - loss: 2.0496 - acc: 0.4618

69339/69338 [==============================] - 11497s 166ms/step - loss: 2.0496 - acc: 0.4618
File ID: 1l9_pwPWox3nW_OuJASYtxHp-sD0q5CCw


In [29]:
model.fit_generator(generator_small(black, batch_size = 1024), steps_per_epoch = num_states(black) / 1024, epochs = 1)
model.save('black1024.h5')
saver('black1024')

Epoch 1/1
17334/17334 [==============================] - ETA: 0s - loss: 2.0357 - acc: 0.4619

17335/17334 [==============================] - 9710s 560ms/step - loss: 2.0357 - acc: 0.4619
File ID: 14yZad6_8-TGsHDPAqYaqNKGyiEMZlXJm


In [30]:
model.fit_generator(generator_small(black, batch_size = 4096), steps_per_epoch = num_states(black) / 4096, epochs = 1)
model.save('black4096.h5')
saver('black4096')

Epoch 1/1
2326/4176 [==============================] - ETA: 1:10:56 - loss: 1.7274 - acc: 0.5132

In [31]:
model.fit_generator(generator_small(white[::-1], batch_size = 64), steps_per_epoch = num_states(white) / 128, epochs = 1)
model.save('white.h5')
saver('white')

133664/133663 [==============================] - 8460s 63ms/step - loss: 1.9380 - acc: 0.4680
File ID: 1dNqWppl7EK6QmlxhTPlUqcKR_iYjRfCX


In [32]:
model.fit_generator(generator_small(white, batch_size = 128), steps_per_epoch = num_states(white) / 128, epochs = 1)
model.save('white.h5')
saver('white')

133664/133663 [==============================] - 13222s 99ms/step - loss: 1.8446 - acc: 0.4883
File ID: 1oZQDbXO-L4LZ4eWkZO2DoH-5eOghCx1k


In [33]:
model.fit_generator(generator_small(white, batch_size = 256), steps_per_epoch = num_states(white) / 256, epochs = 1)
model.save('white256.h5')
saver('white256')

66832/66831 [==============================] - 11055s 165ms/step - loss: 1.8197 - acc: 0.4925
File ID: 10RO8KzLkjJeqU6fm3aEfcwJjGOvfgCOp


In [34]:
model.save('white1024.h5')
saver('white1024')

In [35]:
model.fit_generator(generator_small(white, batch_size = 1024), steps_per_epoch = num_states(white) / 1024, epochs = 1)
model.save('white1024.h5')
saver('white1024')

Epoch 1/1
16707/16707 [==============================] - ETA: 0s - loss: 1.7597 - acc: 0.5058

16708/16707 [==============================] - 9350s 560ms/step - loss: 1.7597 - acc: 0.5058
File ID: 191FOnjZpHzWkgqU2c1eOxuUuC-qqzM6Z


In [36]:
model.fit_generator(generator_small(white, batch_size = 4096), steps_per_epoch = num_states(white) / 4096, epochs = 1)
model.save('white4096.h5')
saver('white4096')

Epoch 1/1
2326/4176 [==============================] - ETA: 1:10:56 - loss: 1.7274 - acc: 0.5132

In [37]:
def upload_file_to_drive(file_name, file_data):
  uploaded = drive.CreateFile({'title': file_name})
  uploaded.SetContentS(file_data)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))
  return uploaded.get('id')

In [39]:
upload_file_to_drive('black.renju', "\n".join(black))
upload_file_to_drive('white.renju', "\n".join(white))

Uploaded file with ID 1NvY2_XsQ-AMfoh06J6p95Kd0878_ReSR
Uploaded file with ID 1CBAGTSBbBUkSYr6RSJH6kJ3rXNOfKyT6


'1CBAGTSBbBUkSYr6RSJH6kJ3rXNOfKyT6'

In [41]:
black = drive.CreateFile({'id': '1NvY2_XsQ-AMfoh06J6p95Kd0878_ReSR'}).GetContentString().split('\n')
white = drive.CreateFile({'id': '1CBAGTSBbBUkSYr6RSJH6kJ3rXNOfKyT6'}).GetContentString().split('\n')

In [42]:
black = np.array(black)
np.save('black.npy', black)
black = 0
white = np.array(white)
np.save('white.npy', white)
white = 0

In [43]:
black[0]

'h8 h9 j10 k10 g11 f10 k9 l8 h11 f11 j8 g10 j9 j7 h10 f12 k8 e12 l7'